In [1]:
import numpy as np
from scipy.optimize import minimize
from Making_datasets_functions import insert_matrix
import imutils
import os
import cv2
import json
import Mask2Base64
import matplotlib.pyplot as plt

In [2]:
from sklearn.cluster import DBSCAN

def optimal_angle(mask, clasterization=False):
    def disp(angle):
        proj = np.sin(angle)*x_proj + np.cos(angle)*y_proj
        return (proj**2).mean()-(proj.mean())**2
    
    indexes = np.nonzero(mask)
    
    if clasterization:
        indexes = np.array(indexes).T
        clustering = DBSCAN(eps=3, min_samples=2).fit(indexes)
        classes = np.array(clustering.labels_)
        counts = np.bincount(classes)
        max_freq = np.argmax(counts)
        indexes = indexes[np.where(classes==max_freq)]
        indexes = indexes.T
        indexes = (list(indexes[0]), list(indexes[1]))
    
    x_proj = np.repeat([np.arange(mask.shape[1])], mask.shape[0], axis=0)[indexes]
    y_proj = np.repeat([np.arange(mask.shape[0])], mask.shape[1], axis=0).T[indexes]
    answer = minimize(disp, x0=[1], bounds=[[-10,10]]).x[0]
    answer += 4*np.pi
    while answer>np.pi/2:
        answer -= np.pi
        
    return answer

In [58]:
def rotate_to_horizontal(data_dir, mask_folder='pred', img_folder='img', res_folder='rotated_img', echo=False):
    mask_files = os.listdir(os.path.join(data_dir, mask_folder))
    for fname in mask_files:
        if echo:
            print("\n", os.path.join(data_dir, mask_folder, fname))
        mask = cv2.imread(os.path.join(data_dir, mask_folder, fname))[:,:,0]
        img_name = fname.split('.')[0]
        image = cv2.imread(os.path.join(data_dir, img_folder, img_name + '.png'))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        angle = optimal_angle(mask, clasterization=True)        #plt.imshow(imutils.rotate(image, angle=-angle/np.pi*180))
        file_name = os.path.join(data_dir, res_folder, img_name + '_rotated' + '.png')
        cv2.imwrite(file_name, imutils.rotate_bound(image, angle=angle/np.pi*180))

In [59]:
DIRECTORY = './cropped/good'
rotate_to_horizontal(DIRECTORY, echo=True)


 ./cropped/good/pred/IMG_1758_ROD_901869481.png

 ./cropped/good/pred/IMG_1751_ROD_901697557.png

 ./cropped/good/pred/IMG_1752_ROD_901700830.png

 ./cropped/good/pred/IMG_1758_ROD_901868787.png

 ./cropped/good/pred/IMG_1750_ROD_901580691.png

 ./cropped/good/pred/IMG_1759_ROD_901727118.png

 ./cropped/good/pred/IMG_6362_ROD_901727727.png

 ./cropped/good/pred/IMG_6365_ROD_901744276.png

 ./cropped/good/pred/IMG_1758_ROD_901869501.png

 ./cropped/good/pred/IMG_1754_ROD_901710542.png

 ./cropped/good/pred/IMG_1749_ROD_901713549.png

 ./cropped/good/pred/IMG_1760_ROD_902819123.png

 ./cropped/good/pred/IMG_1757_ROD_901718733.png

 ./cropped/good/pred/IMG_1760_ROD_902821519.png

 ./cropped/good/pred/IMG_1751_ROD_901693799.png

 ./cropped/good/pred/IMG_1760_ROD_902818996.png

 ./cropped/good/pred/IMG_1760_ROD_902819315.png

 ./cropped/good/pred/IMG_6496_ROD_901780234.png

 ./cropped/good/pred/IMG_6501_ROD_901749629.png

 ./cropped/good/pred/IMG_1752_ROD_901701072.png

 ./cropped/good/pre